# **Carhartの４ファクター:モデル**

## **1. Googleドライブをマウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **2. CSVファイルから株価情報を取得**

In [ ]:
import pandas as pd
import numpy as np

# Googleドライブのパスを指定して読み込み
file_path = '/content/drive/My Drive/csv/input/us_stock_price.csv'

df = pd.read_csv(file_path)

# 'Tikcer'列の値を抽出
ticker_list = df['Ticker'].unique()

# リストを表示
print("Tiker List: ", ticker_list)

# Dateの最大値と最小値を取得
min_date = df['Date'].min()
max_date = df['Date'].max()

print("最小日付:", min_date)
print("最大日付:", max_date)

## **3. 各銘柄の累積リターンを確認**

In [ ]:
import matplotlib.pyplot as plt

# Date列を日付型に変換
df['Date'] = pd.to_datetime(df['Date'])

# 証券Codeごとにグループ化して収益率を計算
df['Simple_Return'] = df.groupby('Ticker')['Price'].pct_change()

# 累積リターンを計算
df['Cumulative_Return'] = (1 + df['Simple_Return']).groupby(df['Ticker']).cumprod()

# Codeごとにグラフを作成
plt.figure(figsize=(10, 6))

for code in df['Ticker'].unique():
    plt.plot(df[df['Ticker'] == code]['Date'], df[df['Ticker'] == code]['Cumulative_Return'], label=code)

# 縦軸を対数スケールに設定
plt.yscale('log')

# グラフのタイトルとラベル
plt.title('Cumulative Return Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Return (log scale)')
plt.legend()

# グラフを表示
plt.show()

## **4. Carhartの4ファクターを読み込み**

In [ ]:
# パスを指定して読み込み
file_path = '/content/drive/My Drive/csv/input/f-f_four_factors.csv'

df_ff = pd.read_csv(file_path)

print(df_ff.head())

## **5. ファクター間の相関、散布図を確認**

In [ ]:
#視覚的に美しいプロットを簡単に作成できるライブラリ
import seaborn as sns

# 最新時点から10年前までのデータを活用
df_ff['Date'] = pd.to_datetime(df_ff['Date']) # Convert 'Date' column to datetime
ten_years_ago = df_ff['Date'].max() - pd.DateOffset(years=10)
df_ff = df_ff[df_ff['Date'] > ten_years_ago]

# ファクターリターンの間の相関を計算
factor_returns = df_ff[['Mkt-RF', 'SMB', 'HML', 'WML']]
correlation_matrix = factor_returns.corr()

# 相関行列を表示
print(correlation_matrix)

# 散布図行列を作成
sns.pairplot(factor_returns)

# グラフを表示
plt.show()

## **6. 個別銘柄のリターンデータとファクターリターンを結合**

In [ ]:
# Date列を日付型に変換
df['Date'] = pd.to_datetime(df['Date'])
df_ff['Date'] = pd.to_datetime(df_ff['Date'])

# データフレームの結合(inner join)
merged_df = pd.merge(df, df_ff, on='Date', how='inner')

# 結果を表示
print(merged_df)

## **7. 証券Tickerごとに回帰分析を実施し、結果をresult_dfに保存しCSVファイルに出力**

In [ ]:
import statsmodels.api as sm

# 証券Tickerごとに回帰分析を実施
tickers = merged_df['Ticker'].unique()

# 結果を格納するためのリスト
result_list = []

for ticker in tickers:
    ticker_data = merged_df[merged_df['Ticker'] == ticker]

    # 説明変数と目的変数を設定
    X = ticker_data[['Mkt-RF', 'SMB', 'HML', 'WML']]
    X = sm.add_constant(X)  # 定数項を追加
    Y = (ticker_data['Simple_Return'] - ticker_data['RF'])

    # 回帰モデルを作成してフィット
    model = sm.OLS(Y, X)
    result = model.fit()

    # 回帰係数とその他の情報を抽出
    for param, value in result.params.items():
        result_list.append({
            'Ticker': ticker,
            'Parameter': param,
            'Coefficient': value,
            'T-Value': result.tvalues[param],
            'P-Value': result.pvalues[param],
            'R-Squared': result.rsquared
        })


# 結果をデータフレームに変換
result_df = pd.DataFrame(result_list)

# CSVファイルに保存
result_df.to_csv('/content/drive/My Drive/csv/output/regression_results.csv', index=False)

# 結果の表示（任意）
print(result_df)
